In [1]:
import tweepy
import csv
import re
import json
import pandas as pd
import numpy as np

In [3]:
ACCESS_TOKEN = '1241442279820349440-YQCAoIPTgl35DsZpy0wE7OSwIjMGGT'
ACCESS_TOKEN_SECRET = 'ZhiceZOw8tQz8wYCLWX9GxTfnHWOaQRoKSwpLEfMV6nzn'
CONSUMER_KEY = "SHxbfc2hZVd2lTgA2yzuXdB7m"
CONSUMER_SECRET = "05HnNsg7r4z49QJJaGUe0n9P0xcZxDJRsXQKV901k6dcbV81Li"


In [13]:
"""
INPUTS:
    consumer_key, consumer_secret, access_token, access_token_secret: codes 
    telling twitter that we are authorized to access this data
    hashtag_phrase: the combination of hashtags to search for
OUTPUTS:
    none, simply save the tweet info to a spreadsheet
"""
def search_for_hashtags(consumer_key, consumer_secret, access_token, access_token_secret, hashtag_phrase):
    
    #create authentication for accessing Twitter
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)

    #initialize Tweepy API
    api = tweepy.API(auth)
    
    #get the name of the spreadsheet we will write to
    fname = '_'.join(re.findall(r"#(\w+)", hashtag_phrase))

    #open the spreadsheet we will write to
    with open('%s.csv' % (fname), 'w') as file:

        w = csv.writer(file)

        #write header row to spreadsheet
        w.writerow(['timestamp', 'tweet_text', 'username', 'all_hashtags', 'followers_count'])

        #for each tweet matching our hashtags, write relevant info to the spreadsheet
        for tweet in tweepy.Cursor(api.search, q=hashtag_phrase+' -filter:retweets', lang="en", tweet_mode='extended').items(1):
            # line for checking json fetch
#             print(json.dumps(tweet._json, indent=3))
            w.writerow([tweet.created_at, tweet.full_text.replace('\n',' '), tweet.user.screen_name, [e['text'] for e in tweet._json['entities']['hashtags']], tweet.user.followers_count])

In [14]:
hashtag_phrase = '#blacklivesmatter'

if __name__ == '__main__':
    search_for_hashtags(CONSUMER_KEY, CONSUMER_SECRET, ACCESS_TOKEN, ACCESS_TOKEN_SECRET, hashtag_phrase)

{
   "created_at": "Mon Jun 15 11:25:03 +0000 2020",
   "id": 1272490297990287365,
   "id_str": "1272490297990287365",
   "full_text": "Covid-19, Climate Justice, and #BlackLivesMatter #Fridays4Future #ClimateStrikeOnline https://t.co/8PBOG70Jan via @MarianAllen https://t.co/gxr325rU4m",
   "truncated": false,
   "display_text_range": [
      0,
      126
   ],
   "entities": {
      "hashtags": [
         {
            "text": "BlackLivesMatter",
            "indices": [
               31,
               48
            ]
         },
         {
            "text": "Fridays4Future",
            "indices": [
               49,
               64
            ]
         },
         {
            "text": "ClimateStrikeOnline",
            "indices": [
               65,
               85
            ]
         }
      ],
      "symbols": [],
      "user_mentions": [
         {
            "screen_name": "MarianAllen",
            "name": "Marian Allen",
            "id": 20601055,
          

In [ ]:
"""
INPUTS:
    consumer_key, consumer_secret, access_token, access_token_secret: codes 
    telling twitter that we are authorized to access this data
    hashtag_phrase: the combination of hashtags to search for
OUTPUTS:
    none, simply save the tweet info to a spreadsheet
"""
def search_for_hashtags(CONSUMER_KEY, CONSUMER_SECRET, ACCESS_TOKEN, ACCESS_TOKEN_SECRET, hashtag_phrase, n):
    
    #create authentication for accessing Twitter
    auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
    auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

    #initialize Tweepy API
    api = tweepy.API(auth)
    
    
    hash_tags_to_plot = pd.DataFrame(columns=['hashtags'])
    empty = []
    #get the name of the spreadsheet we will write to
    for tweet in tweepy.Cursor(api.search, q=hashtag_phrase+' -filter:retweets', lang="en", tweet_mode='extended').items(n):
#         print(json.dumps(tweet._json, indent=2, sort_keys=True))
        print(len(tweet._json['full_text']), end='\n\n')
        twitter_hashtags = []
        
        twitter_hashtags.append([e['text'] for e in tweet._json['entities']['hashtags']])
        for x in twitter_hashtags:
            for y in x:
                empty.append(y.lower())
    hash_tags_to_plot['hashtags'] = empty
    hash_tags_to_plot.to_csv(f'{hashtag_phrase}.csv')
    return hash_tags_to_plot

In [ ]:
hashtag_phrase = '#blacklivesmatter'

if __name__ == '__main__':
    search_for_hashtags(CONSUMER_KEY, CONSUMER_SECRET, ACCESS_TOKEN, ACCESS_TOKEN_SECRET, hashtag_phrase, 50)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as ex
# ax = plt.figure(figsize=(20, 30))
# ax = sns.barplot(y='index', x='hashtags', data=df['hashtags'].value_counts().reset_index())


import plotly.graph_objects as go

fig = go.Figure([go.Bar(y=df['hashtags'].value_counts().sort_values(ascending=True).reset_index()['index'], x=df['hashtags'].value_counts().sort_values(ascending=True).reset_index()['hashtags'], orientation='h')])
fig.update_layout(title_text='Hashtag Frequency',
                  xaxis_tickfont_size=14,
                  xaxis=dict(
                            title='Count Mentions',
                            titlefont_size=16,
                            tickfont_size=14),
                  yaxis=dict(
                            title='Hashtag',
                            titlefont_size=10,
                            tickfont_size=6,
                            ),
                 autosize=True,
                 height=1000,
                    margin=dict(
                        l=50,
                        r=50,
                        b=100,
                        t=100,
                        pad=4
                    ))
fig.show()